<a href="https://colab.research.google.com/github/anshupandey/ms-generativeai-apr2025/blob/main/code4B_Prompt_Engineering_with_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering Techniques

In [1]:
from openai import OpenAI
client = OpenAI()

In [2]:
# creating a function to get outcome
def generate_response(prompt,model="gpt-4o",temperature=0):
  messages = [{"role":"user","content":prompt}]
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature=temperature
  )
  return response.choices[0].message.content


## Zero Shot Prompting

In [3]:
# zero shot prompting
prompt = f"""
Create a conversation story between child and grandparent.
"""
response = generate_response(prompt)
print(response)

**Title: The Secret Garden Adventure**

**Characters:**
- Lily, the curious child
- Grandpa Joe, the wise and loving grandparent

---

**Scene: A sunny afternoon in Grandpa Joe's backyard**

**Lily:** Grandpa, can we go on an adventure today?

**Grandpa Joe:** An adventure, you say? Well, I think I might have just the thing. Have you ever heard of the secret garden?

**Lily:** A secret garden? No, I haven't! Where is it?

**Grandpa Joe:** It's right here in our backyard, hidden away from the world. But only those with the keenest eyes and the bravest hearts can find it.

**Lily:** Wow! Can we go there now? I want to see it!

**Grandpa Joe:** Of course, my little explorer. But first, we need to gather some supplies. Every adventurer needs a trusty backpack. Let's see... a flashlight, a magnifying glass, and some snacks. You never know what we might encounter.

**Lily:** I've got my backpack ready, Grandpa! What's next?

**Grandpa Joe:** Follow me, Lily. The entrance to the secret garden

## Few Shot Prompting

Here, the model is given a few examples (shots) to guide its response. By providing context or previous instances, the model can better understand and generate the desired output. For example, showing a model several examples of translated sentences before asking it to translate a new one.


In [4]:
# few shot prompting
prompt = f"""
your task is to answer in a consistent sytle:

<child>: Teach me about patience.
<grandma>: Patience is like waiting for whole day to see the moon in the evening and then sleeping after having a look at it.

<child>: Teach me about resilience.
"""
response = generate_response(prompt)
print(response)

<grandma>: Resilience is like a tree that bends with the wind but never breaks, standing tall again once the storm has passed.


#### Few shot prompting with role setup

In [5]:
result = client.chat.completions.create(
    model='gpt-4o',  # e.g. gpt-35-instant
    max_tokens=200,
    temperature=0.9,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "That was an awesome experience"},
        {"role": "assistant", "content": "positive"},
        {"role": "user", "content": "I won't do that again"},
        {"role": "assistant", "content": "negative"},
        {"role": "user", "content": "That was not worth my time"},
        {"role": "assistant", "content": "negative"},
        {"role": "user", "content": "You can't miss this"}
    ],
)
print(result.choices[0].message.content)

positive


## Chain of Thought Prompting

In [6]:
# Without Chain of Thought Prompting
prompt = f"""

Teacher: Johan had 5 apples, he bought 4 more boxes with 5 apples each. How many apples does he have now?
Student: 25 apples

Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

To solve this problem, we need to follow the steps:

1. Marry initially had 5 pens.
2. She gave 3 pens to her friend, so she has \(5 - 3 = 2\) pens left.
3. She then bought 2 boxes of pens, with 3 pens in each box. This means she bought \(2 \times 3 = 6\) pens.
4. Adding the pens she bought to the pens she had left, Marry now has \(2 + 6 = 8\) pens.

So, Marry has 8 pens now.


In [8]:
# with Chain of Thought prompting
prompt = f"""

Teacher: Johan had 5 apples, he bought 4 more boxes with 5 apples each. How many apples does he have now?
Student: Johan had 5 apples, 4 boxes with 5 apples each = 5 + 4*5 = 25 apples

Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

Marry initially had 5 pens. She gave 3 pens to her friend, so she had:

5 - 3 = 2 pens remaining.

Then, she bought 2 boxes with 3 pens in each box, which means she got:

2 * 3 = 6 pens.

Adding the pens she bought to the pens she had left:

2 + 6 = 8 pens.

So, Marry has 8 pens now.


In [9]:
# with Chain of Thought prompting
prompt = f"""

Think step by step to answer below question, do not come to conclusion without following process.
Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

To solve this problem, let's break it down step by step:

1. **Initial Number of Pens:**
   - Marry initially had 5 pens.

2. **Pens Given Away:**
   - She gave 3 pens to her friend.
   - So, the number of pens she has after giving away is:  
     \( 5 - 3 = 2 \) pens.

3. **Pens Bought:**
   - She bought 2 boxes of pens, with each box containing 3 pens.
   - The total number of pens in the boxes is:  
     \( 2 \times 3 = 6 \) pens.

4. **Total Number of Pens Now:**
   - Add the pens she has after giving away to the pens she bought:  
     \( 2 + 6 = 8 \) pens.

Therefore, Marry has 8 pens now.


## Tree of Thoughts (ToT)

Tree of Thoughts (ToT) prompting is a framework that generalizes over chain-of-thought prompting and encourages exploration over thoughts that serve as intermediate steps for general problem-solving with language models.


How does it work?

- ToT prompting breaks problems down into smaller parts, similar to CoT prompting, but goes further by combining this with the ability to explore multiple solution paths in parallel, forming a tree instead of a single chain. Each thought is generated or solved independently and passed to the next step, allowing the model to self-evaluate and decide whether to continue with that path or choose another.

In [10]:

prompt = """
Imagine 5 different experts are answering this question.
They will brainstorm the answer step by step reasoning carefully and taking all facts into consideration
All experts will write down 1 step of their thinking,
then share it with the group.
They will each critique their response, and the all the responses of others
They will check their answer based on science and the laws of physics
They will keep going through steps until they reach their conclusion taking into account the thoughts of the other experts
If at any time they realise that there is a flaw in their logic they will backtrack to where that flaw occurred
If any expert realises they're wrong at any point then they acknowledges this and start another train of thought
Each expert will assign a likelihood of their current assertion being correct
Continue until the experts agree on the single most likely location
The question is.
1. Carlos is at the swimming pool.
2. He walks to the locker room, carrying a towel.
3. He puts his watch in the towel and carries the towel tightly to a lounger at the poolside.
4. At the lounger he opens and vigorously shakes the towel, then walks to the snack bar.
5. He leaves the towel at the snack bar, then walks to the diving board.
6. Later Carlos realises he has has lost his watch. Where is the single most likely location of the watch?
"""


ans=generate_response(prompt)

print(ans)

**Step 1: Initial Thoughts**

**Expert 1:** Carlos likely lost his watch when he shook the towel at the lounger. The vigorous shaking could have dislodged the watch. Probability: 70%

**Expert 2:** The watch might have fallen out when Carlos carried the towel to the lounger. The act of carrying it tightly could have caused it to slip out. Probability: 60%

**Expert 3:** The watch could be at the snack bar. Carlos left the towel there, and the watch might have fallen out when he put the towel down. Probability: 50%

**Expert 4:** It's possible the watch is still in the towel. Carlos might not have noticed it when he left the towel at the snack bar. Probability: 40%

**Expert 5:** The watch could have fallen out at any point between the locker room and the diving board. Probability: 30%

**Step 2: Critique and Discussion**

**Expert 1:** I still believe the shaking at the lounger is the most likely point of loss. However, I acknowledge that the watch could have fallen out earlier.

**Exp

## Chain-of-Verification Prompting

The Chain-of-Verification (CoVe) prompt engineering method aims to reduce hallucinations through a verification loop.

In [11]:
Question="Name some athletes who were born in United states"

prompt=f'''Here is the question: {Question}.

First, generate a response.

Then, create and answer verification questions based on this response to check for accuracy. Think it through and make sure you are extremely accurate based on the question asked.

After answering each verification question, consider these answers and revise the initial response to formulate a final, verified answer. Ensure the final response reflects the accuracy and findings from the verification process.'''



ans=generate_response(prompt)

print(ans)


Initial Response:
Some athletes who were born in the United States include Michael Jordan, Serena Williams, Tom Brady, Simone Biles, and LeBron James.

Verification Questions and Answers:

1. Was Michael Jordan born in the United States?
   - Yes, Michael Jordan was born in Brooklyn, New York, United States.

2. Was Serena Williams born in the United States?
   - Yes, Serena Williams was born in Saginaw, Michigan, United States.

3. Was Tom Brady born in the United States?
   - Yes, Tom Brady was born in San Mateo, California, United States.

4. Was Simone Biles born in the United States?
   - Yes, Simone Biles was born in Columbus, Ohio, United States.

5. Was LeBron James born in the United States?
   - Yes, LeBron James was born in Akron, Ohio, United States.

Revised Final Response:
After verifying the information, it is confirmed that all the athletes mentioned were indeed born in the United States. Therefore, some athletes who were born in the United States include Michael Jordan

# Model Limitations: Hallucinations

In [12]:
# asking for a specific format
text = """
Washing clothes without washing machines is easy!
First you need to get some warm water. Mix the warm water with some detergent. Second you can put the dirty clothes in it and let it stay wet for some time.
During this period the stains will get loose and mixed with water.
thereafter you can take clothes out, rub the dirt/stains with a soft brush to remove the stains if any.
get the clothes mixed with water to get the detergent out of it. and then put it for drying on a rope.
Enjoy the clean clothes.
"""

prompt = f"""
you will be provided with some text delimited by triple backticks. Rewrite those into a set of instructions in the following format:

Step 1: ...
Step 2: ....
..
Step N:...

```{text}
```
"""
response = generate_response(prompt)
print(response)


Step 1: Get some warm water.  
Step 2: Mix the warm water with some detergent.  
Step 3: Put the dirty clothes in the water and let them soak for some time.  
Step 4: Allow the stains to loosen and mix with the water.  
Step 5: Take the clothes out and rub the dirt/stains with a soft brush to remove any remaining stains.  
Step 6: Rinse the clothes with water to remove the detergent.  
Step 7: Hang the clothes on a rope to dry.  
Step 8: Enjoy the clean clothes.  


In [13]:
# asking for a specific format
text = """
THe morning today is amazing, the sun is looking bright in between clouds, the weather is pleasent and the wind is mindblowing. on a whole i am loving the day today.
"""

prompt = f"""
you will be provided with some text delimited by triple backticks. Rewrite those into a set of instructions in the following format:

Step 1: ...
Step 2: ....
..
Step N:...

```{text}
```
"""
response = generate_response(prompt)
print(response)


```
Step 1: Observe the morning and note that it is amazing.

Step 2: Notice the sun shining brightly between the clouds.

Step 3: Feel the pleasant weather.

Step 4: Enjoy the mind-blowing wind.

Step 5: Appreciate and love the day as a whole.
```


In [14]:
# asking for a specific format
text = """
Washing clothes without washing machines is easy!
First you need to get some warm water. Mix the warm water with some detergent. Second you can put the dirty clothes in it and let it stay wet for some time.
During this period the stains will get loose and mixed with water.
thereafter you can take clothes out, rub the dirt/stains with a soft brush to remove the stains if any.
get the clothes mixed with water to get the detergent out of it. and then put it for drying on a rope.
Enjoy the clean clothes.
"""

prompt = f"""
you will be provided with some text delimited by triple backticks.
Rewrite those into a set of instructions in the following format only if contains a sequence of instructions:

Step 1: ...
Step 2: ....
..
Step N:...

```{text}
```

if the text does not contain a sequence of instructions, then simply write 'No steps provided'
"""
response = generate_response(prompt)
print(response)


Step 1: Get some warm water.  
Step 2: Mix the warm water with some detergent.  
Step 3: Put the dirty clothes in the water and let them stay wet for some time.  
Step 4: Take the clothes out and rub the dirt/stains with a soft brush to remove any stains.  
Step 5: Rinse the clothes with water to remove the detergent.  
Step 6: Put the clothes for drying on a rope.  
Step 7: Enjoy the clean clothes.


In [15]:
# asking for a specific format
text = """
THe morning today is amazing, the sun is looking bright in between clouds, the weather is pleasent and the wind is mindblowing. on a whole i am loving the day today.
"""

prompt = f"""
you will be provided with some text delimited by triple backticks.
Rewrite those into a set of instructions in the following format only if contains a sequence of instructions:

Step 1: ...
Step 2: ....
..
Step N:...

```{text}
```

if the text does not contain a sequence of instructions, then simply write 'No steps provided'
"""
response = generate_response(prompt)
print(response)


No steps provided


In [16]:
prompt = f"""
Tell me about Ergonamic chair procuced and sold by Morning Star IT Services.
"""

generate_response(prompt)

'As of my last update in October 2023, there is no specific information available about an "Ergonamic chair" produced and sold by Morning Star IT Services. It\'s possible that this product is either a new release or a niche product that hasn\'t been widely documented in the sources I have access to. \n\nIf you\'re interested in this chair, I recommend checking Morning Star IT Services\' official website or contacting them directly for the most accurate and up-to-date information. They would be able to provide details on the chair\'s features, pricing, availability, and any other relevant information.'

In [17]:
prompt = f"""
Tell me about Ergonamic Toothbrush by colgate.
"""

generate_response(prompt)

'As of my last update, Colgate has not released a product specifically named the "Ergonamic Toothbrush." However, Colgate is well-known for producing a variety of toothbrushes that focus on ergonomic design to enhance user comfort and improve oral hygiene. These toothbrushes often feature:\n\n1. **Ergonomic Handles**: Designed to fit comfortably in the hand, reducing strain during brushing and allowing for better control.\n\n2. **Soft Bristles**: Many Colgate toothbrushes come with soft bristles to effectively clean teeth and gums without causing irritation.\n\n3. **Advanced Bristle Design**: Some models include multi-level or angled bristles to reach different areas of the mouth more effectively.\n\n4. **Special Features**: Certain toothbrushes may include tongue cleaners, cheek cleaners, or polishing cups for a more comprehensive cleaning experience.\n\n5. **Electric Options**: Colgate also offers electric toothbrushes with ergonomic designs, providing features like timers and differ

In [18]:
prompt = f"""
Tell me about Ultraslim Toothbrush by MorningStar.
"""

generate_response(prompt)

'As of my last update in October 2023, I don\'t have specific information about an "Ultraslim Toothbrush" by a company named MorningStar. It\'s possible that this product is either very new, niche, or not widely covered in the sources I have access to. \n\nIf this is a recent product release, I recommend checking the company\'s official website or their press releases for the most accurate and up-to-date information. Additionally, online retailers or consumer review sites might have user reviews and more details about the product\'s features and benefits.'

In [20]:
prompt = f"""
Generate one line description for each of the following city:
1. Manila
2. Delhi
3. MorningStarCity
"""

print(generate_response(prompt))

1. Manila: The bustling capital of the Philippines, known for its rich history, vibrant culture, and diverse culinary scene.
2. Delhi: A dynamic metropolis in India, blending ancient landmarks with modern urban life and renowned for its street food and markets.
3. MorningStarCity: A fictional or lesser-known city, potentially characterized by its unique charm and emerging cultural or economic significance.


In [21]:
prompt = f"""
Generate one line description for each of the following city, check if the city exists, if it does not exist, say no description can be generated:
1. Manila
2. Delhi
3. MorningStarCity
"""

print(generate_response(prompt))

1. Manila: The bustling capital of the Philippines, known for its rich history, vibrant culture, and diverse culinary scene.
2. Delhi: The capital city of India, offering a blend of ancient landmarks, bustling markets, and modern urban life.
3. MorningStarCity: No description can be generated, as this city does not exist.


## Conversational App

In [22]:
# creating a function to get outcome
def generate_response(messages,model='gpt-4o'):
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature=0.5
  )
  return response.choices[0].message.content


In [23]:
messages = [{"role":"system","content":"You are a helpful assistant"},]
while True:
    uinput = input("You: ")
    print("You: ",uinput)
    if uinput.lower() in ['q','quit','exit']:
        break
    messages.append({"role":"user","content":uinput})
    response = generate_response(messages=messages)
    print(f"Bot: {response}")
    messages.append({"role":"assistant","content":response})


You:  Hi My name is ANshu, I live in Mumbai. 
Bot: Hello Anshu! It's nice to meet you. How can I assist you today?
You:  tell me about the city I live in .
Bot: Certainly! Mumbai, formerly known as Bombay, is the capital city of the Indian state of Maharashtra. It's the most populous city in India and one of the most densely populated cities in the world. Here are some key points about Mumbai:

1. **Economic Hub**: Mumbai is the financial, commercial, and entertainment capital of India. It is home to major financial institutions like the Reserve Bank of India, the Bombay Stock Exchange, and numerous multinational corporations. The city contributes significantly to India's GDP.

2. **Bollywood**: Mumbai is synonymous with Bollywood, the Hindi-language film industry. It produces a large number of films each year and is a major cultural influence in India and beyond.

3. **Cultural Diversity**: The city is a melting pot of cultures, with a diverse population that includes people from all 

In [24]:
messages

[{'role': 'system', 'content': 'You are a helpful assistant'},
 {'role': 'user', 'content': 'Hi My name is ANshu, I live in Mumbai. '},
 {'role': 'assistant',
  'content': "Hello Anshu! It's nice to meet you. How can I assist you today?"},
 {'role': 'user', 'content': 'tell me about the city I live in .'},
 {'role': 'assistant',
  'content': "Certainly! Mumbai, formerly known as Bombay, is the capital city of the Indian state of Maharashtra. It's the most populous city in India and one of the most densely populated cities in the world. Here are some key points about Mumbai:\n\n1. **Economic Hub**: Mumbai is the financial, commercial, and entertainment capital of India. It is home to major financial institutions like the Reserve Bank of India, the Bombay Stock Exchange, and numerous multinational corporations. The city contributes significantly to India's GDP.\n\n2. **Bollywood**: Mumbai is synonymous with Bollywood, the Hindi-language film industry. It produces a large number of films 

## Thank You